## Day 3: Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

    17  16  15  14  13
    18   5   4   3  12
    19   6   1   2  11
    20   7   8   9  10
    21  22  23---> ...

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

* Data from square 1 is carried 0 steps, since it's at the access port.
* Data from square 12 is carried 3 steps, such as: down, left, left.
* Data from square 23 is carried only 2 steps: up twice.
* Data from square 1024 must be carried 31 steps.

How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle input is 325489.

In [6]:
import math

INPUT = 325489

In [7]:
def width(num):
    estimate = math.ceil(math.sqrt(num))
    if estimate % 2 == 0:
        estimate += 1
    return estimate

assert width(1) == 1
assert width(12) == 5
assert width(23) == 5

In [22]:
def center(table):
    return int(len(table) / 2), int(len(table) / 2)


def walk(table):
    """Start in the middle, spiral outwards, and yield coordinates"""
    step = 0

    # Start in the middle
    x, y = center(table)
    yield y, x

    # One iteration is one trip around the existing square
    for _ in range(int(len(table) / 2)):
        step += 2

        # step right to the next square
        x += 1
        yield y, x

        # to the top
        for yoff in range(step - 1):
            y -= 1
            yield y, x

        # to the left
        for xleft in range(step):
            x -= 1
            yield y, x

        # to the bottom
        for _ in range(step):
            y += 1
            yield y, x

        # to the right
        for _ in range(step):
            x += 1
            yield y, x


def construct(num):
    # Allocate space for the table and fill with zeroes
    row = [0] * width(num)
    # To copy a list in Python, a slice from start to end is necessary
    # With row, and not row[:], you'd get the same row
    table = [row[:] for _ in row]

    i = 1

    # Fill the table with values
    for x, y in walk(table):
        table[x][y] = i
        if i == num:
            point = x, y
        i += 1

    return (table, point)

In [23]:
construct(10)

([[17, 16, 15, 14, 13],
  [18, 5, 4, 3, 12],
  [19, 6, 1, 2, 11],
  [20, 7, 8, 9, 10],
  [21, 22, 23, 24, 25]],
 (3, 4))

In [24]:
def locate(number, table):
    for x, y in walk(table):
        if table[x][y] == number:
            return x, y

def compute_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def distance(number):
    table, point = construct(number)
    origin = center(table)
    #point = locate(number, table)
    return compute_distance(origin, point)

In [25]:
assert distance(1) == 0
assert distance(12) == 3
assert distance(23) == 2
assert distance(1024) == 31

In [26]:
distance(INPUT)

552

## Part Two

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

* Square 1 starts with the value 1.
* Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
* Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
* Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
* Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

    147  142  133  122   59
    304    5    4    2   57
    330   10    1    1   54
    351   11   23   25   26
    362  747  806--->   ...

What is the first value written that is larger than your puzzle input?

In [30]:
def cellv(table, x, y, default=0):
    """
    Return cell value if coordinates are within bounds
    Return default value if coordinates are out of bounds
    """
    val = default
    
    if 0 <= x <= len(table):
        if 0 <= y <= len(table[x]):
            val = table[x][y]
    
    return val
    
    
def neighbors(t, x, y):
    """
    Return values of neighbor cells (zero if out of bounds)
    """
    neigh = [
        [cellv(t, x-1, y-1), cellv(t, x-1, y), cellv(t, x-1, y+1)],
        [cellv(t,   x, y-1), cellv(t,   x, y), cellv(t,   x, y+1)],
        [cellv(t, x+1, y-1), cellv(t, x+1, y), cellv(t, x+1, y+1)],
    ]
    return neigh


def sum_neighbors(table, x, y):
    """Return sum of all neighbor cells"""
    return sum(sum(row) for row in neighbors(table, x, y))


def construct2(num):
    # Similar tricks as in construct(num)
    row = [0] * width(num)
    table = [row[:] for _ in row]

    # Initialize the center of the table
    x, y = center(table)
    table[x][y] = 1  # init
    
    for x, y in walk(table):
        nsum = sum_neighbors(table, x, y)
        table[x][y] = nsum
        
        if nsum > num:
            print("A winner is: {nsum} ({x},{y})".format(**locals()))
            break
        
    return table

In [31]:
construct2(10)

A winner is: 11 (3,1)


[[0, 0, 0, 0, 0],
 [0, 5, 4, 2, 0],
 [0, 10, 1, 1, 0],
 [0, 11, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [32]:
_ = construct2(INPUT)

A winner is: 330785 (281,284)
